<a href="https://colab.research.google.com/github/Sebanthalas/fem-dnn/blob/main/Test_FEM_solver_JAN23_NSE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
try:
    import dolfin
except ImportError:
    !wget "https://fem-on-colab.github.io/releases/fenics-install.sh" -O "/tmp/fenics-install.sh" && bash "/tmp/fenics-install.sh"
    import dolfin
%cd drive/MyDrive/research
import platform, sys
python_version=platform.python_version()
from distutils.version import LooseVersion, StrictVersion
import matplotlib.pyplot as plt;
from IPython.display import clear_output, display; import time; import dolfin.common.plotting as fenicsplot 
import time
import os, sys, shutil
from google.colab import files
#!python -m pip install Tasmanian --user
!pip install hdf5storage
!

In [ ]:
%cd drive/MyDrive/research
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, regularizers
# Other libraries
import scipy.io as sio
from fenics import *
#import Tasmanian
import numpy as np
import time
import matplotlib.pyplot as plt
from numpy import linalg as la
import time, os, argparse, io, shutil, sys, math, socket
import scipy.io as sio
from dolfin import *
import sympy2fenics as sf
import random

In [ ]:

# ==============================================================
# The sympy2fenics.py file is needed to compute in a simple way:
# - derivatives
# - tensor products
# ==============================================================
def my_LK_fun(x,z,d):
  pi = 3.14159265359
  pi_s = pi
  L_c = 1.0/8.0
  L_p = np.max([1.0, 2.0*L_c])
  L_c_s = L_c
  L_p_s = L_p
  L = L_c/L_p
  L_s = L
  string = -1.2+sqrt(sqrt( pi_s )*  L_s /2.0)* z[0]
  for j in range(2, input_dim):
    term = z[j-1]*sqrt(sqrt(pi_s)*L_s )*exp(-pow(np.floor(j/2.0)* pi*L_s,2.0)/8.0)
    if j % 2 == 0:
      term = term*sin(np.floor(j/2.0)* pi_s*x/L_p_s)
    else:
      term = term*cos(np.floor(j/2.0)* pi_s*x/L_p_s)
    string = string + term
  string = np.exp(  string )
  return(string) 
def str2exp(s):
    return sf.sympy2exp(sf.str2sympy(s))
def coeff_extr(j):
    #This exctracts the coefficient of the different spaces
    # j is the index of the space:
    # j=o the vector space for uh
    # j=1 the space for  component of te11
    # j=2 the space for  component of te12
    # j=3 the space for  component of te21
    # j=4 the first  row of sigma
    # j=5 the second row of sigma
    # j=6 the lagrange multiplier (not uch important)
    # For this to work the nature of the space has to be the same
    W  = Function(Hh)
    #Getting the exact DOF location
    DoF_map   = Hh.sub(j).dofmap()
    DoF_index = DoF_map.dofs()
    AUX1 = W.vector().get_local()    # empty DoF vector 
    AUX2 = Usol.vector().get_local() # All DoF
    AUX1[DoF_index] = AUX2[DoF_index]                # corresponding assignation
    W.vector().set_local(AUX1)       # corresponding assignation to empy vector
    coeff_vector = np.array(W.vector().get_local()) 
    return coeff_vector
# ==============================================================
# Boundary conditions
# ==============================================================
class MyExpression(UserExpression):
  def eval(self, value, x):
    if x[1] >= 1- DOLFIN_EPS:#-my_LK_fun(x[0],z,d):
      value[0] = 0.0
    elif x[1] <= 0.0+ DOLFIN_EPS:#+my_LK_fun(x[0],z,d):
      value[0] = 0.0
    elif x[0] > 1- DOLFIN_EPS:
      value[0] = ( 1.0 - ( abs(abs(x[1]) - 0.5 ))/(0.5)  ) 
    elif x[0] < 0.0+ DOLFIN_EPS:
      value[0] = ( 1.0 - ( abs(abs(x[1]) - 0.5 ))/(0.5)  ) 
    elif ( (x[0] > 0.0625 - DOLFIN_EPS) and (x[0] < 0.1875 +DOLFIN_EPS) and (x[1] > 0.4375 - DOLFIN_EPS) and (x[1] < 0.5625 + DOLFIN_EPS) ):
      value[0] = 0.0   
    else:
      value[0] = 0.0
  def value_shape(self):
    return (1,)

# ==============================================================
# The following gives the informations about the solver. 
# ==============================================================
parameters["form_compiler"]["representation"] = "uflacs"
parameters["form_compiler"]["cpp_optimize"] = True
parameters["form_compiler"]["quadrature_degree"] = 4
list_linear_solver_methods()
# ==============================================================
# Define the parameters for the PDE and SG points
# ==============================================================
# All parameters
default_parameters = {
  'trial_': 1,
	'num_train_samples':1, #20
	'num_test_samples': 0, #50
  'mesh_op': 2, # choose mesh refiniment, 1,2,3,4. Default=2
	'FE_degree': 1, # Use P1 for 2D or above
  'example' : 'other',
  'input_dim': 4
}
params = default_parameters
exact_FE_soln = 0
using_SG_points =0
#random samples
_trial = params['trial_']
np_seed = _trial
np.random.seed(np_seed)


#================================================================
# Choose the mesh: there is 4  barycentric refininements. 
#                  - use plot(mesh) to see them.
#================================================================
nk = params['mesh_op']
example = params['example']
input_dim = params['input_dim']
meshname="meshes/obstac%03g.xml"%nk
mesh = Mesh(meshname)
nn   = FacetNormal(mesh)
#================================================================
# only for errors
#================================================================
hvec = mesh.hmax(); # pick the h_max from the mesh
nvec = 0;           # Same number as K
#================================================================
# The following for is not necessary. I use it when I want to obtain results
# for different values of m
#================================================================

for m in [1]:
    All_Train_coeff   = []
    All_Train_coeff_p = []
    All_Test_coeff    = []
    All_Test_coeff_p  = []
    _L4unorm= []
    _L2tnorm= []
    _L2pnorm= []
    _L2snorm= []
    input_dim = params['input_dim']
    Nu = m
    d = input_dim       # parametric dimension
    y_in_train = 2.0*np.random.rand(d,m) - 1.0  # training points    
    Re = Constant(150)
    g     = Constant((0.,0.))
    f     = Constant((0.0,0.))
    cero  = Constant((0.,0.))
    upper = Constant((10.,0.))
    Id    = Constant(((1.,0.),(0.,1.)))
    #y_in_train = (2.0*np.random.rand(d,m) - 1.0)
    # Generate the training data
    for i in range(m):
      coeff_each_m = []
      coeff_each_m_p = []
      # time
      t_start = time.time()
      # Paramters
      z = y_in_train[:,i]
      # ******* Exact solutions for error analysis ****** 
      #================================================================
      # The viscosity of the fluid depends on the parameters
      #================================================================
      
      if example == 'logKL_expansion':
         pi = 3.14159265359
         pi_s = str(pi)
         L_c = 1.0/8.0
         L_p = np.max([1.0, 2.0*L_c])
         L_c_s = str(L_c)
         L_p_s = str(L_p)
         L = L_c/L_p
         L_s = str(L)      
         string = '-4.5+sqrt(sqrt(' + pi_s + ')*' + L_s + '/2.0)*' + str(z[0])
         for j in range(2, input_dim):
           term = str(z[j-1]) + '*sqrt(sqrt(' + pi_s + ')*' + L_s + ')*exp(-pow(floor('
           term = term + str(j) + '/2.0)*' + pi_s + '*' + L_s + ',2.0)/8.0)' 
           if j % 2 == 0:
             term = term + '*sin(floor(' + str(j) + '/2.0)*' + pi_s + '*x/' + L_p_s + ')'
           else:
             term = term + '*cos(floor(' + str(j) + '/2.0)*' + pi_s + '*x/' + L_p_s + ')'
           string = string + '+' + term
         string = 'exp(' + string + ')'
      else:# use a simple example where only the first two dimensions have x dependence
        string = 1/150
        #string = '5.0+exp(' + str(z[0]) + '+' + str(z[1]) + '+' + str(z[2]) + '+'+ str(z[3]) +' )' 
        
      #mu = Expression(str2exp(string), degree=2, domain=mesh)
      #mu = Constant(string)
      mu = Constant(1/0.1)
      #================================================================
      # Boundary condition
      #================================================================ 
      u_D = MyExpression()
      u_str = '(  1.0, 0.0 )'       
      u_ex     = Expression(str2exp(u_str), degree=7, domain=mesh)     
      
    #================================================================
    #  *********** Finite Element spaces ************* #
    #================================================================
      deg= params['FE_degree']
      Pkv = VectorElement('DG', mesh.ufl_cell(), deg)
      Pk  = FiniteElement('DG', mesh.ufl_cell(), deg)
      RTv = FiniteElement('RT', mesh.ufl_cell(), deg+1)
      R0  = FiniteElement('R', mesh.ufl_cell(), 0)
      Hh  = FunctionSpace(mesh, MixedElement([Pkv,Pk,Pk,Pk,RTv,RTv,R0]))
      Ph  = FunctionSpace(mesh,'CG',1)
      nvec = Hh.dim()
     #================================================================
     # *********** Trial and test functions ********** #
     #================================================================
      Utrial = TrialFunction(Hh)
      Usol   = Function(Hh)
      W_trainsol   = Function(Hh)
      u, t11, t12, t21, Rsig1, Rsig2, xi= split(Usol)
      v, s11, s12, s21, Rtau1, Rtau2, ze= TestFunctions(Hh)
      t = as_tensor(((t11,t12),(t21,-t11)))
      s = as_tensor(((s11,s12),(s21,-s11))) 
      sigma = as_tensor((Rsig1,Rsig2))
      tau   = as_tensor((Rtau1,Rtau2))
      # ********** Boundary conditions ******** #
      # All Dirichlet BCs become natural in this mixed form 
      # *************** Variational forms ***************** #
      #================================================================
      # flow equations
      #================================================================   
      # Weak formulation  
      BigA = - dot(div(sigma),v)*dx + 0.5*dot(t*u,v)*dx
      BigB = 2.*mu*inner(sym(t),s)*dx - 0.5*inner(dev(outer(u,u)),s)*dx - inner(dev(sigma),s)*dx 
      BigC = inner(tau,t)*dx  + dot(u,div(tau))*dx
      # right part and boundary equation
      F  = dot(g+f,v)*dx
      G  = (dot(tau*nn,u_ex*u_D[0]))*ds 
      # zero average of trace
      Z  = (tr(2*sigma+outer(u,u))) * ze * dx + tr(tau) * xi * dx
      #Stiffness matrix
      FF = BigA + BigB + BigC  - F - G  + Z
    	# solver						    
      Tang = derivative(FF, Usol, Utrial)
      solve(FF == 0, Usol, J=Tang)
      #Solutions
      uh, t11h, t12h, t21h, Rsigh1, Rsigh2, xih = Usol.split()
      th=as_tensor(((t11h,t12h),(t21h,-t11h)))					    
      sigmah = as_tensor((Rsigh1,Rsigh2))        
      # dimension-dependent (not separating H_0(div) with c*I)
      #getting coeffiecients of p
      ph = project(-0.25*tr(2*sigmah+outer(uh,uh)),Ph)
      coeff_each_m_p = ph.vector().get_local()
      All_Train_coeff_p.append(coeff_each_m_p)
      #
      num_subspaces = W_trainsol.num_sub_spaces()-1
      for j in range(num_subspaces):
        coef_one_trial = coeff_extr(j)
        coeff_each_m.append(coef_one_trial)
      All_Train_coeff.append(coeff_each_m)
      print('====================================================================')
      print('i = ', i, ' ', y_in_train[:,i])
      print('====================================================================')
      L4unorm  = sqrt(sqrt(assemble( ((uh)**2)**2*dx)))
      L2tnorm  = sqrt(assemble( (th)**2*dx ))
      L2pnorm  = sqrt(assemble((ph)**2*dx) ) 
      L2snorm  = sqrt(assemble((sigmah)**2*dx) ) 
      print(L4unorm,L2tnorm,L2pnorm)
      _L4unorm.append(L4unorm)
      _L2tnorm.append(L2tnorm)
      _L2pnorm.append(L2pnorm)
      _L2snorm.append(L2snorm)

     
      if i == 0:
        K = len(All_Train_coeff[0][0])
        print('FE degrees of freedom K = ',K)
      if exact_FE_soln:
        # Compute error in L4 and L2 norms
        erroru_L4 = sqrt(sqrt(assemble( ((u_ex-uh)**2)**2*dx)))
        errort_L2 = sqrt(assemble((th-t_ex)**2*dx))
        errorp_L2 = sqrt(assemble((ph-p_ex)**2*dx)) 
        errorp_Hdiv4_3 = sqrt(assemble((sigma_ex-sigmah)**2*dx) ) +  pow(  assemble(  ( pow(  ( div(sigma_ex)-div(sigmah)  )**2,2.0/3.0 )**1*dx)  )  , 3.0/4.0 )   
        # Compute maximum error at vertices
        vertex_values_u_ex= u_ex.compute_vertex_values(mesh)
        vertex_values_uh   = uh.compute_vertex_values(mesh)
        error_max         = np.max(np.abs(vertex_values_u_ex - vertex_values_uh))
        timer =time.time() - t_start
        # Print errors
        print('i = ', i, ' ', y_in_train[:,i])
        print('====================================================================')
        print(' h    &   e(u)     &  e(t)   &    e(p)   &    e(si)   &   e_m(u) &   time ')
        print('====================================================================')
        print('%5.5g     %4.4g    %4.4g    %4.4g      %4.4g       %4.4g     %4.4g  ' % (hvec, erroru_L4, errort_L2, errorp_L2, errorp_Hdiv4_3, error_max,timer ))
        print('======================================================================')
    print('Generated inputs of size: ', y_in_train.shape)
    print('Generated outputs of size: ', coef_one_trial.shape[0])
    # generate the test points
    if using_SG_points:
        # create the sparse grid generator
        grid = Tasmanian.SparseGrid()
        # specify the level    
        level = 4
        # generate sparse grid points and weights
        grid.makeGlobalGrid(d, 0, level, "level", "clenshaw-curtis")
        # get the points and weights from the generator
        y_in_test = np.transpose(grid.getPoints())
        w_test_weights = grid.getQuadratureWeights()
        m_test = y_in_test.shape[1]
        print('Using Clenshaw-Curtis sparse grid points with ', m_test, ' points')
        print('Sum of weights = ', np.sum(w_test_weights))
        # scatter plot the points
        #plt.scatter(y_in_test[0,:], y_in_test[1,:])
        #plt.show()

    else:
        # get the number of test points
        m_test = params['num_test_samples']

        # generate the points randomly
        y_in_test = 2.0*np.random.rand(d,m_test) - 1.0

    print('Generating the testing data')
    # Generate the training data
    for i in range(m_test):
      coeff_each_m = []
      coeff_each_m_p = []
      t_start = time.time()
      # get the training data inputs 
      z = y_in_test[:,i]
      #================================================================
      # The viscosity of the fluid depends on the parameters
      #================================================================
      mu = Expression(str2exp(string), degree=2, domain=mesh)
      #================================================================
      # Exact solutions settings
      #================================================================        
      u_D = MyExpression()    
      u_ex     = Expression(str2exp(u_str), degree=7, domain=mesh)   
    #================================================================
    #  *********** Finite Element spaces ************* #
    #================================================================
      Pkv = VectorElement('DG', mesh.ufl_cell(), deg)
      Pk  = FiniteElement('DG', mesh.ufl_cell(), deg)
      RTv = FiniteElement('RT', mesh.ufl_cell(), deg+1)
      R0  = FiniteElement('R', mesh.ufl_cell(), 0)
      Hh  = FunctionSpace(mesh, MixedElement([Pkv,Pk,Pk,Pk,RTv,RTv,R0]))
      Ph  = FunctionSpace(mesh,'CG',1)
      nvec = Hh.dim()
     #================================================================
     # *********** Trial and test functions ********** #
     #================================================================
      Utrial = TrialFunction(Hh)
      Usol   = Function(Hh)
      W_trainsol = Function(Hh)
      u, t11, t12, t21, Rsig1, Rsig2, xi= split(Usol)
      v, s11, s12, s21, Rtau1, Rtau2, ze= TestFunctions(Hh)
      t = as_tensor(((t11,t12),(t21,-t11)))
      s = as_tensor(((s11,s12),(s21,-s11))) 
      sigma = as_tensor((Rsig1,Rsig2))
      tau   = as_tensor((Rtau1,Rtau2))
      # ********** Boundary conditions ******** #
      # All Dirichlet BCs become natural in this mixed form 
      # *************** Variational forms ***************** #
      #================================================================
      # flow equations
      #================================================================   
      # Weak formulation  
      BigA = - dot(div(sigma),v)*dx + 0.5*dot(t*u,v)*dx
      BigB = 2.*mu*inner(sym(t),s)*dx - 0.5*inner(dev(outer(u,u)),s)*dx - inner(dev(sigma),s)*dx 
      BigC = inner(tau,t)*dx  + dot(u,div(tau))*dx
      # right part and boundary equation
      F  = dot(g+f,v)*dx
      G  = (dot(tau*nn,u_ex*u_D[0]))*ds
      # zero average of trace
      Z  = (tr(2*sigma+outer(u,u))) * ze * dx + tr(tau) * xi * dx
        
      FF = BigA + BigB + BigC  - F - G  + Z
    							    
      Tang = derivative(FF, Usol, Utrial)

      solve(FF == 0, Usol, J=Tang)

      uh, t11h, t12h, t21h, Rsigh1, Rsigh2, xih = Usol.split()
      th=as_tensor(((t11h,t12h),(t21h,-t11h)))					    
      sigmah = as_tensor((Rsigh1,Rsigh2))        
      # dimension-dependent (not separating H_0(div) with c*I)
      
      ph = project(-0.25*tr(2*sigmah+outer(uh,uh)),Ph)
      coeff_each_m_p = ph.vector().get_local()
      All_Test_coeff_p.append(coeff_each_m_p)
      # Get the evaluations in a testing grid
      num_subspaces = W_trainsol.num_sub_spaces()-1
      coeff_each_m = Usol.vector().get_local()
      All_Test_coeff.append(coeff_each_m)
      print('====================================================================')
      print('i = ', i, ' ', y_in_test[:,i])
      print('====================================================================')

      if i == 0:
        K = len(All_Train_coeff[0][0])
        print('FE degrees of freedom K = ',K)
      if exact_FE_soln:
        # Compute error in L2 norm
        #erroru_L4 = sqrt(sqrt(assemble( ((u_ex-uh)**2)**2*dx)))
        #errort_L2 = sqrt(assemble((th-t_ex)**2*dx))
        #errorp_L2 = sqrt(assemble((ph-p_ex)**2*dx)) 
        #errorp_Hdiv4_3 = sqrt(assemble((sigma_ex-sigmah)**2*dx) ) +  pow(  assemble(  ( pow(  ( div(sigma_ex)-div(sigmah)  )**2,2.0/3.0 )**1*dx)  )  , 3.0/4.0 )     
        # Compute maximum error at vertices
        vertex_values_u_ex= u_ex.compute_vertex_values(mesh)
        vertex_values_uh   = uh.compute_vertex_values(mesh)
        error_max         = np.max(np.abs(vertex_values_u_ex - vertex_values_uh))
        timer =time.time() - t_start
        # Print errors
        print('i = ', i, ' ', y_in_test[:,i])
        print('====================================================================')
        print('h    &   e(u)     &  e(t)   &    e(p)   &    e(si)   &   e_m(u) &   time ')
        print('====================================================================')
        print('%5.5g     %4.4g    %4.4g    %4.4g      %4.4g       %4.4g     %4.4g  ' % (hvec, erroru_L4, errort_L2, errorp_L2, errorp_Hdiv4_3, error_max,timer ))
        print('======================================================================')
    # Save the training and test#
    train_scratchdir = '/content/drive/My Drive/research/TRAIN/DNN_BANACH_NSE_REAL_d'+str(d)+'_trial_'+str(_trial)+'/result_m_'+str(Nu)+''
    testn_scratchdir = '/content/drive/My Drive/research/TESTN/DNN_BANACH_NSE_REAL_d'+str(d)+'_trial_'+str(_trial)+'/result_m_'+str(Nu)+''
    result_folder  = train_scratchdir + '/results_mesh' +str(nk)+ '_FE_degree_'+ str(deg)+'_no_p'
    scratch_folder = testn_scratchdir + '/results_mesh' +str(nk)+ '_FE_degree_'+ str(deg)+'_no_p'
    if not os.path.exists(result_folder):
        os.makedirs(result_folder)    
    if not os.path.exists(scratch_folder): 
        os.makedirs(scratch_folder)

    print('Generated inputs of size: ', y_in_test.shape)
    print('Generated outputs of size: ', All_Train_coeff[0][0].shape)

    run_data = {}
    run_data['d']              = d
    run_data['K']              = K
    run_data['Nu']             = Nu
    run_data['mesh_op']        = nk
    run_data['FE_degree']      = deg
    run_data['m']              = m
    run_data['m_test']         = m_test

 
   #===============================================
   # DOF Coeff- for each function separate
   #===============================================    

    run_data['All_Train_coeff']     = All_Train_coeff
    run_data['All_Test_coeff']      = All_Test_coeff
    run_data['All_Train_coeff_p']     = All_Train_coeff_p
    run_data['All_Test_coeff_p']     = All_Test_coeff_p
    run_data['y_in_test_data']      = y_in_test
    run_data['y_in_train_data']      = y_in_train
    run_data['_L4unorm']      = _L4unorm
    run_data['_L2tnorm']      = _L2tnorm
    run_data['_L2pnorm']      = _L2pnorm
    run_data['_L2snorm']      = _L2snorm
    #run_data['w_test_weights']      = w_test_weights

    run_data['fenics_mesh_coords']       = np.array(mesh.coordinates())
    run_data['fenics_mesh_cells']        = np.array(mesh.cells())
    run_data['fenics_mesh_num_cells']    = np.array(mesh.num_cells())
    run_data['fenics_mesh_num_edges']    = np.array(mesh.num_edges())
    run_data['fenics_mesh_num_vertices'] = np.array(mesh.num_vertices())
    run_data['fenics_mesh_hmax']         = np.array(mesh.hmax())
    run_data['fenics_mesh_hmin']         = np.array(mesh.hmin())
    run_data['fenics_mesh_rmax']         = np.array(mesh.rmax())
    run_data['fenics_mesh_rmin']         = np.array(mesh.rmin())
    sio.savemat(result_folder + '/run_data_train.mat', run_data)